In [2]:
from CNTools.datasets import load_ct_order, prep_crc, Dataset
from CNTools.identification import CC, CFIDF, CNE, Spatial_LDA
from CNTools.smoothing import NaiveSmooth, HMRF
import os
import pandas as pd
import pickle

In [16]:
# Download CRC, T2D, and HLT data
os.system('gdown --folder https://drive.google.com/drive/folders/1eC3ahSIzCZ5vq_bynSl3IX7V9w_bQgg8?usp=share_link')

In [3]:
df = prep_crc(pd.read_csv('data/CRC/CRC_clusters_neighborhoods_markers.csv'))
ct_order = load_ct_order('data/CRC/ct_order.json')
ds = Dataset(df, ct_order)

In [18]:
# identifier = CFIDF(n_cns=9, eps=32, r=0.8)
identifier = CNE(n_cns=9, perp=15, lam=0.25)
# identifier = Spatial_LDA(n_cns=9, eps=50, b=0.025)

cns = identifier.fit(ds)

100%|██████████| 35/35 [00:03<00:00,  9.89it/s]


Entropy: 2.605, Size: 34.52


In [19]:
smoother = NaiveSmooth(ds=ds, n_cns=identifier.n_cns, feats=identifier.feats, s=3)
cns_smoothed_naive = smoother.fit(cns)

Entropy: 2.624, Size: 48.08


In [20]:
smoother = HMRF(ds=ds, n_cns=identifier.n_cns, eps=40, beta=9)
cns_smoothed_hmrf = smoother.fit(cns)

Entropy: 2.674, Size: 37.82


In [ ]:
df.to_csv('data/CRC/CRC_df.csv', index=False)
pickle.dump(ds, open(f'data/CRC/CRC_ds.pkl', 'wb'))
pickle.dump(cns, open(f'cn/CRC/CNE/cns_n_cns=9_perp=15_lam=0.25.pkl', 'wb'))
pickle.dump(cns_smoothed_naive, open(f'cn/CRC/CNE/cns_n_cns=9_perp=15_lam=0.25_naive.pkl', 'wb'))
pickle.dump(cns_smoothed_hmrf, open(f'cn/CRC/CNE/cns_n_cns=9_perp=15_lam=0.25_hmrf.pkl', 'wb'))